In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [7]:
print(f"Number of GPU: {torch.cuda.device_count()}")
print (f"GPU Name: {torch.cuda.get_device_name(0)}")

Number of GPU: 1
GPU Name: NVIDIA GeForce MX150


In [8]:
# Tentukan perangkat yang digunakan (GPU jika tersedia, jika tidak maka CPU)
device_cpu = torch.device("cpu")
device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
device_cpu

device(type='cpu')

In [10]:
device_gpu

device(type='cuda')

In [11]:
# Pastikan CUDA tersedia
if torch.cuda.is_available():
    # Mendapatkan jumlah GPU yang tersedia
    num_gpus = torch.cuda.device_count()
    
    # Menampilkan informasi setiap GPU
    for i in range(num_gpus):
        print(f"--- GPU {i} ---")
        print(f"Nama GPU: {torch.cuda.get_device_name(i)}")
        print(f"Memori Total: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
        print(f"Memori Alokasi: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
        print(f"Memori Tersedia: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")
        print(f"MultiProcessor Count: {torch.cuda.get_device_properties(i).multi_processor_count}")
else:
    print("CUDA tidak tersedia, pastikan perangkat memiliki GPU yang mendukung CUDA.")


--- GPU 0 ---
Nama GPU: NVIDIA GeForce MX150
Memori Total: 2.00 GB
Memori Alokasi: 0.00 GB
Memori Tersedia: 0.00 GB
MultiProcessor Count: 3


In [12]:
# Definisikan transformasi untuk data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [13]:
# Load dataset MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [19]:
image, label = trainset[5]
label

2

In [10]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [11]:
# Definisikan model sederhana
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
# Fungsi untuk training
def train_model(device, trainloader):
    model = SimpleNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    start_time = time.time()
    for epoch in range(1):  # Menggunakan 1 epoch untuk komparasi cepat
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

    end_time = time.time()
    print(f'Training completed in: {end_time - start_time:.2f} seconds on {device}')


In [18]:
# Latih model di CPU
print("Training on CPU:")
train_model(device_cpu, trainloader)

Training on CPU:
[Epoch 1, Batch 100] loss: 1.286
[Epoch 1, Batch 200] loss: 0.464
[Epoch 1, Batch 300] loss: 0.378
[Epoch 1, Batch 400] loss: 0.326
[Epoch 1, Batch 500] loss: 0.295
[Epoch 1, Batch 600] loss: 0.256
[Epoch 1, Batch 700] loss: 0.256
[Epoch 1, Batch 800] loss: 0.254
[Epoch 1, Batch 900] loss: 0.221
Training completed in: 67.57 seconds on cpu


In [19]:
# Latih model di GPU (jika tersedia)
if torch.cuda.is_available():
    print("\nTraining on GPU:")
    train_model(device_gpu, trainloader)
else:
    print("\nGPU is not available, skipping GPU training.")


Training on GPU:
[Epoch 1, Batch 100] loss: 1.429
[Epoch 1, Batch 200] loss: 0.482
[Epoch 1, Batch 300] loss: 0.407
[Epoch 1, Batch 400] loss: 0.354
[Epoch 1, Batch 500] loss: 0.318
[Epoch 1, Batch 600] loss: 0.287
[Epoch 1, Batch 700] loss: 0.256
[Epoch 1, Batch 800] loss: 0.244
[Epoch 1, Batch 900] loss: 0.232
Training completed in: 70.01 seconds on cuda
